In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os, re, sys
from sklearn.model_selection import train_test_split
import skimage.io

In [2]:
# test if GPUs are available
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# set project root, maybe you need to firstly 
# add shortcut of CS 766 Project to drive.
project_root = './drive/MyDrive/CS 766 Project/Project Coding and Data Files'

In [5]:
# class to initialize CNNs
class OriginCNN(object):
  """docstring for OriginCNN"""
  def __init__(self):
    # self.optimizer = 'sgd'
    self.optimizer = tf.keras.optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
    self.loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # will add more properties


  def build(self):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu', input_shape=(1000, 1000, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='sigmoid'))
    model.add(layers.Dense(5))

    model.summary()
    model.compile(optimizer=self.optimizer,
                loss=self.loss,
                metrics=['accuracy'])
    self.model = model


  def train(self, train_images, train_labels, validate_images, validate_labels, epochs):
    self.history = self.model.fit(train_images, train_labels, epochs=epochs, 
        validation_data=(validate_images, validate_labels))


  def evaluate(self, test_images, test_labels):
    # plt.plot(self.history.history['accuracy'], label='accuracy')
    # plt.plot(self.history.history['val_accuracy'], label = 'val_accuracy')
    # plt.xlabel('Epoch')
    # plt.ylabel('Accuracy')
    # plt.ylim([0.5, 1])
    # plt.legend(loc='lower right')
    # test_loss, test_acc = self.model.evaluate(test_images, test_labels, verbose=2)
    # print(test_acc)
    _, test_acc = self.model.evaluate(test_images, test_labels, verbose=2)
    return test_images.shape[0], test_acc

  def save_model(self, filepath):
    self.model.save(filepath)

  def load_model(self, filepath):
    self.model = tf.keras.models.load_model(filepath)

In [6]:
class DR_resized(object):
  """docstring for DR_resized"""
  def __init__(self, is_training, batch_id):
    if(is_training):
      # load images
      all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Processed_Xtrain_Batch%d.npy" % batch_id))
      # add a dimension for channels
      all_images = np.expand_dims(all_images, axis=-1)

      # load labels
      all_labels = np.load(os.path.join(project_root, "Data Batches", "Ytrain_Batch%d.npy" % batch_id), allow_pickle=True)
      all_labels = np.array([item[1] for item in all_labels])
      all_labels = np.expand_dims(all_labels, axis=-1)
      
      self.train_images, self.validate_images, self.train_labels, self.validate_labels \
      = train_test_split(all_images, all_labels, test_size=0.2, random_state=9876)
    else:
        # load images
      all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Processed_Xtest_Batch%d.npy") % batch_id)
      # add a dimension for channels
      all_images = np.expand_dims(all_images, axis=-1)

      # load labels
      all_labels = np.load(os.path.join(project_root, "Data Batches", "Ytest_Batch%d.npy" % batch_id), allow_pickle=True)
      all_labels = np.array([item[1] for item in all_labels])
      all_labels = np.expand_dims(all_labels, axis=-1)
      
      self.test_images = all_images
      self.train_labels = all_labels


In [7]:
# initialize CNN
myModel = OriginCNN()
myModel.build()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 333, 333, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 166, 166, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 55, 55, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0

In [ ]:
# # training process
# file_num = 9
# epoch_num = 5
# for i in range(file_num):
#   myData = DR_resized(True, i)
#   print("data batch %d loaded" % i)
#   myModel.train(myData.train_images, myData.train_labels, myData.validate_images, myData.validate_labels, epoch_num)
#   print("data batch %d trained" % i)
#   # del myData
#   myData = None

In [8]:
# training process, manually
epoch_num = 5
myData = DR_resized(True, 0)
print("data batch %d loaded" % 0)
myModel.train(myData.train_images, myData.train_labels, myData.validate_images, myData.validate_labels, epoch_num)
print("data batch %d trained" % 0)
# del myData
myData = None

data batch 0 loaded
Epoch 1/5
78/78 [==============================] - 38s 75ms/step - loss: 0.9400 - accuracy: 0.7278 - val_loss: 0.8529 - val_accuracy: 0.7452
Epoch 2/5
78/78 [==============================] - 5s 66ms/step - loss: 0.8500 - accuracy: 0.7451 - val_loss: 0.8670 - val_accuracy: 0.7452
Epoch 3/5
78/78 [==============================] - 5s 67ms/step - loss: 0.9103 - accuracy: 0.7192 - val_loss: 0.8503 - val_accuracy: 0.7452
Epoch 4/5
78/78 [==============================] - 5s 67ms/step - loss: 0.9167 - accuracy: 0.7163 - val_loss: 0.8508 - val_accuracy: 0.7452
Epoch 5/5
78/78 [==============================] - 5s 67ms/step - loss: 0.8858 - accuracy: 0.7324 - val_loss: 0.8464 - val_accuracy: 0.7452
data batch 0 trained


In [14]:
myData

In [15]:
# training process, manually
epoch_num = 5
myData = DR_resized(True, 1)
print("data batch %d loaded" % 1)
myModel.train(myData.train_images, myData.train_labels, myData.validate_images, myData.validate_labels, epoch_num)
print("data batch %d trained" % 1)
# del myData
myData = None

data batch 1 loaded
Epoch 1/5
78/78 [==============================] - 5s 68ms/step - loss: 0.8798 - accuracy: 0.7330 - val_loss: 0.8481 - val_accuracy: 0.7452
Epoch 2/5
78/78 [==============================] - 5s 67ms/step - loss: 0.8765 - accuracy: 0.7330 - val_loss: 0.8456 - val_accuracy: 0.7452
Epoch 3/5
78/78 [==============================] - 5s 67ms/step - loss: 0.8776 - accuracy: 0.7330 - val_loss: 0.8496 - val_accuracy: 0.7452
Epoch 4/5
78/78 [==============================] - 5s 67ms/step - loss: 0.8756 - accuracy: 0.7330 - val_loss: 0.8574 - val_accuracy: 0.7452
Epoch 5/5
78/78 [==============================] - 5s 67ms/step - loss: 0.8732 - accuracy: 0.7330 - val_loss: 0.8577 - val_accuracy: 0.7452
data batch 1 trained


In [16]:
myData.train_images = None
myData.validate_images = None

AttributeError: ignored

In [ ]:
# training process, manually
epoch_num = 5
myData = DR_resized(True, 2)
print("data batch %d loaded" % 2)
myModel.train(myData.train_images, myData.train_labels, myData.validate_images, myData.validate_labels, epoch_num)
print("data batch %d trained" % 2)
# del myData
myData = None

In [ ]:
# training process, manually
epoch_num = 5
myData = DR_resized(True, 3)
print("data batch %d loaded" % 3)
myModel.train(myData.train_images, myData.train_labels, myData.validate_images, myData.validate_labels, epoch_num)
print("data batch %d trained" % 3)
# del myData
myData = None

In [ ]:
# training process, manually
epoch_num = 5
myData = DR_resized(True, 4)
print("data batch %d loaded" % 4)
myModel.train(myData.train_images, myData.train_labels, myData.validate_images, myData.validate_labels, epoch_num)
print("data batch %d trained" % 4)
# del myData
myData = None

In [ ]:
# testing process
file_num = 2
num_list = []
acc_list = []
for i in range(file_num):
  myData = DR_resized(False, i)
  num, acc = myModel.evaluate(myData.test_images, myData.test_labels)
  print(num, acc)
  num_list.append(num)
  acc_list.append(acc)
  del myData

In [ ]:
# compute overall accuracy
num_list = np.array(num_list)
acc_list = np.array(acc_list)
acc_overall = (num_list * acc_list).sum() / num_list.sum()
print(acc_overall)

In [ ]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
myModel.save_model(os.path.join(project_root, "Trained Models", dt_string))

INFO:tensorflow:Assets written to: ./drive/MyDrive/CS 766 Project/Project Coding and Data Files/Trained Models/tmp/assets


In [ ]:
myModel.load_model(os.path.join(project_root, "Trained Models", "tmp"))